In [1]:
import numpy as np
import plotly.graph_objects as go
from scipy.integrate import odeint
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import widgets, interactive_output

In [2]:
mumax_Glc_aerob = 0.5  # h^-1
mumax_Glc_crab = 0.34  # h^-1
mumax_Glc_anaerob = 0.25  # h^-1
mumax_Eth_aerob = 0.36  # h^-1
kS_Glc_aerob = 0.1  # kg/m^3
kS_Glc_crab = 0.001  # kg/m^3
kS_Glc_anaerob = 0.1  # kg/m^3
kS_O2_aerob = 30  # Vol-%
kS_O2_crab = 30  # Vol-%
kS_Eth_aerob = 0.1  # kg/m^3
kI_Glc_aerob = 0.1  # kg/m^3
kI_Eth_aerob = 46  # kg/m^3
kI_Eth_crab = 46  # kg/m^3
kI_O2_anaerob = 10  # Vol-%
T1 = 0.2  # h
T2 = 0.2  # h
T3 = 0.2  # h
T4 = 0.2  # h
rd = 0.005  # h^-1
Y_XGlc_aerob = 0.56  # g/g
Y_XGlc_crab = 0.14  # g/g
Y_XGlc_anaerob = 0.15  # g/g
Y_XEth_aerob = 0.67  # g/g
Y_CO2Glc_aerob = 1.46  # g/g
Y_CO2Glc_crab = 0.53  # g/g
Y_CO2Glc_anaerob = 0.48  # g/g
Y_CO2Eth_aerob = 1.9  # g/g
Y_EthGlc_anaerob = 0.51  # g/g
Y_EthGlc_crab = 0.31  # g/g
RQ_Glc_aerob = 1  # g/g
RQ_Glc_crab = 9  # g/g
RQ_Eth_aerob = 0.66  # g/g
Y_H2OGlc_aerob = 6  # mol/mol
Y_H2OGlc_crab = 0.25  # mol/mol
Y_H2OEth_aerob = 3  # mol/mol
Y_QGlc_aerob = 15.6e-3  # J/kg
Y_QGlc_crab = 1.08e-3  # J/kg
Y_QGlc_anaerob = 0.46e-3  # J/kg
Y_QEth_aerob = 29.7e-3  # J/kg
KF_rx = 1 #placeholder for temperature-dependent correction factor

In [3]:
# Create widgets 
c_Eth_0_widget = widgets.FloatSlider(value=0, min=0, max=100, step=0.1, description='ethanol (g/L)')
p_CO2_0_widget = widgets.FloatSlider(value=1, min=0, max=100, step=0.1, description='CO2 (g/L)')
p_O2_0_widget = widgets.FloatSlider(value=20, min=0, max=100, step=0.1, description='O2 (Vol-%)')
m_H20_0_widget = widgets.FloatSlider(value=0, min=0, max=100, step=0.1, description='Initial ethanol(g/L)')
Q_0_widget = widgets.FloatSlider(value=0, min=0, max=10, step=0.1, description='Temperature')
KR_E_0_widget = widgets.FloatSlider(value=0, min=0, max=100, step=0.1, description='half-saturation ethanol')
KR_O2_0_widget = widgets.FloatSlider(value=0, min=0, max=10, step=0.1, description='half-saturation O2 ')
Xd_0_widget = widgets.FloatSlider(value=0, min=0, max=10, step=0.1, description='Dead cell mass(g)')
Xv_0_widget = widgets.FloatSlider(value=5, min=0, max=100, step=0.1, description='Inoculum (g)')
c_Glc_0_widget = widgets.FloatSlider(value=5, min=0, max=10, step=0.1, description='Substrate (g/L)')

# Observe other widgets
def update_parameters(change):
    global c_Eth_0, p_CO2_0, p_O2_0, m_H20_0, Q_0, KR_E_0, KR_O2_0, Xd_0, t, c_Glc_0
    c_Eth_0 = c_Eth_0_widget.value
    p_CO2_0 = p_CO2_0_widget.value
    p_O2_0 = p_O2_0_widget.value
    m_H20_0 = m_H20_0_widget.value
    Q_0 = Q_0_widget.value
    KR_E_0 = KR_E_0_widget.value
    KR_O2_0 = KR_O2_0_widget.value
    Xd_0 = Xd_0_widget.value
    Xv_0 = Xv_0_widget.value
    c_Glc_0 = c_Glc_0_widget.value
    
# Update other parameters as needed
c_Eth_0_widget.observe(update_parameters, names='value')
p_CO2_0_widget.observe(update_parameters, names='value')
p_O2_0_widget.observe(update_parameters, names='value')
m_H20_0_widget.observe(update_parameters, names='value')
Q_0_widget.observe(update_parameters, names='value')
KR_E_0_widget.observe(update_parameters, names='value')
KR_O2_0_widget.observe(update_parameters, names='value')
Xd_0_widget.observe(update_parameters, names='value')
Xv_0_widget.observe(update_parameters, names='value')
c_Glc_0_widget.observe(update_parameters, names='value')

In [4]:
def Crab_Tree(y, t):
    Xv, Xd, c_Glc, c_Eth, p_CO2, p_O2, m_H20, Q, KR_O2, KR_E = y

    # Anaerobic
    mu_Glc_anaerob = mumax_Glc_anaerob*c_Glc/(kS_Glc_anaerob+c_Glc)*KR_O2 # (5) Anaerobic
    q_Glc_anaerob = 1/Y_XGlc_anaerob*mu_Glc_anaerob # (10) specific substrate uptake rate
    q_CO2_Glc_anaerob = Y_CO2Glc_anaerob*q_Glc_anaerob*(1-Y_XGlc_anaerob) # (17) Carbon Dioxide Production rate
    qp_Eth_Glc_anaerob = Y_EthGlc_anaerob*q_Glc_anaerob*(1-Y_XGlc_anaerob) # (15) Ethanol Production rate
    q_Q_Glc_anaerob = Y_QGlc_anaerob*q_Glc_anaerob*(1-Y_XGlc_anaerob) # (26) Heat Generation
    
    # Inhibition
    mu_Eth_aerob = mumax_Eth_aerob*c_Eth/(kS_Eth_aerob+c_Eth)*p_O2/(kS_O2_aerob+p_O2)*KR_E # (4) Aerobic inhibited by ehtanol 
    q_Eth_aerob = 1/Y_XEth_aerob*mu_Eth_aerob # (11) specific substrate uptake rate
    q_CO2_Eth_aerob = Y_CO2Eth_aerob*q_Eth_aerob*(1-Y_XEth_aerob) # (19) Carbon Dioxide Production rate
    q_O2_Eth_aerob = q_CO2_Eth_aerob*32/(44*RQ_Eth_aerob) # (22) Oxygen uptake rate 
    q_H2O_Eth_aerob = Y_H2OEth_aerob*q_Eth_aerob*(1-Y_XEth_aerob) # (25) Water Production rate
    q_Q_Eth_aerob = Y_QEth_aerob*q_Eth_aerob*(1-Y_XEth_aerob) # (29) Heat Generation
     
    # Crab_Tree
    mu_Glc_crab = mumax_Glc_crab*c_Glc/(kS_Glc_crab+c_Glc)*p_O2/(kS_O2_crab+p_O2)*kI_Eth_crab/(kI_Eth_crab+c_Eth)*(1-np.exp(-t/T2))*SW_Glc_crab # (3) Crab tree
    q_Glc_crab = 1/Y_XGlc_crab*mu_Glc_crab # (8) specific substrate uptake rate
    q_CO2_Glc_crab = Y_CO2Glc_crab*q_Glc_crab*(1-Y_XGlc_crab) # (16) Carbon Dioxide Production rate
    q_O2_Glc_crab = q_CO2_Glc_crab*32/(44*RQ_Glc_crab) # (21) Oxygen uptake rate 
    q_H2O_Glc_crab = Y_H2OGlc_crab*q_Glc_crab*(1-Y_XGlc_crab) # (24) Water Production rate
    qp_Eth_Glc_crab = Y_EthGlc_crab*q_Glc_crab*(1-Y_XGlc_crab) # (14) Ethanol Production rate
    q_Q_Glc_crab = Y_QGlc_crab*q_Glc_crab*(1-Y_XGlc_crab) # (27) Heat Generation
    
    # Aerobic
    mu_Glc_aerob = mumax_Glc_aerob * c_Glc / (kS_Glc_aerob + c_Glc) * p_O2 / (kS_O2_aerob + p_O2) * kI_Eth_aerob / (kI_Eth_aerob + c_Eth) * (1 - np.exp(-t / T1)) # (4) aerobic
    q_Glc_aerob = 1/Y_XGlc_aerob*mu_Glc_aerob # (9) specific substrate uptake rate
    q_CO2_Glc_aerob = Y_CO2Glc_aerob*q_Glc_aerob*(1-Y_XGlc_aerob) # (18) Carbon Dioxide Production rate
    q_O2_Glc_aerob = q_CO2_Glc_aerob*32/(44*RQ_Glc_aerob) # (20) Oxygen uptake rate 
    q_H2O_Glc_aerob = Y_H2OGlc_aerob*q_Glc_aerob*(1-Y_XGlc_aerob) # (23) Water Production rate
    q_Q_Glc_aerob = Y_QGlc_aerob*q_Glc_aerob*(1-Y_XGlc_aerob) # (28) Heat Generation
        
    mu_total = (mu_Glc_aerob + mu_Glc_anaerob + mu_Glc_crab + mu_Eth_aerob) * KF_rx - rd # (30) Total biomass growth
    q_Glc_total = (q_Glc_aerob + q_Glc_anaerob + q_Glc_crab) * KF_rx # (31) Total Glucose Consumption rate
    q_Eth_total = (q_Eth_aerob - qp_Eth_Glc_anaerob - qp_Eth_Glc_crab) * KF_rx # (32) Total Ethanol Production rate
    q_CO2_total = (q_CO2_Glc_aerob + q_CO2_Glc_anaerob + q_CO2_Glc_crab + q_CO2_Eth_aerob) * KF_rx # (33)Total CO2 Production rate
    q_O2_total = (q_O2_Glc_aerob + q_O2_Glc_crab + q_O2_Eth_aerob) * KF_rx # (34) Total O2 Consumption rate
    q_H2O_total = (q_H2O_Glc_aerob + q_H2O_Glc_crab + q_H2O_Eth_aerob) * KF_rx # (35) Total Water Production rate
    q_Q_total = 1/3600 * (q_Q_Glc_aerob + q_Q_Glc_anaerob + q_Q_Glc_crab + q_Q_Eth_aerob) * KF_rx # (36) Total Heat Production rate

    dydt = [
        mu_total * Xv,
        rd * Xv,
        -q_Glc_total * Xv,
        -q_Eth_total * Xv,
        q_CO2_total * Xv,
        -q_O2_total * Xv,
        0 * q_O2_total * Xv,
        q_Q_total * Xv,
        1 / T3 * (kI_O2_anaerob / (kI_O2_anaerob + p_O2) - KR_O2) * (kI_Eth_aerob / (kI_Eth_aerob + c_Eth) - KR_E),
        0
    ]
    return dydt


In [5]:
def create_plot(tspan, c_Glc, c_Eth, XvXd, p_CO2, p_O2):
    # Create plots using Plotly
    fig = go.Figure()
    
    fig.add_trace(go.Scatter(x=tspan * 60, y=c_Glc, name='Glucose', visible=True , line=dict(color='grey')))
    fig.add_trace(go.Scatter(x=tspan * 60, y=c_Eth, name='Ethanol', visible=True , line=dict(color='orange')))
    fig.add_trace(go.Scatter(x=tspan * 60, y=Xv + Xd, name='Cell mass', visible=True , line=dict(color='green')))
    fig.add_trace(go.Scatter(x=tspan * 60, y=p_O2, name='Oxygen', visible=True , line=dict(color='blue')))
    fig.add_trace(go.Scatter(x=tspan * 60, y=p_CO2, name='Carbondioxide', visible=True , line=dict(color='red')))
    
    # Define buttons for the left position
    buttons_left = [
    {'args': [None, {'frame': {'duration': 150, 'redraw': False},
                    'fromcurrent': True, "transition": {"duration": 150, 'easing': 'quadratic-in-out'}}],
     'label': 'Play',
     'method': 'animate'},
    {'args': [[None], {'frame': {'duration': 150, 'redraw': False}, 'mode': 'immediate'}],
     'label': 'Pause',
     'method': 'animate'},
    {'args': [None, {'frame': {'duration': 150, 'redraw': True},
                    'fromcurrent': False, "transition": {"duration": 150, 'easing': 'quadratic-in-out'}}],
     'label': 'Reset',
     'method': 'animate'},]

    # Define buttons for the top position
    buttons_top = [
        dict(label="All",
          method="update",
          args=[{"visible": [True, True, True, True, True]},
               {"title": "All",
               "annotations": []}]),
        dict(label="Glucose",
             method="update",
             args=[{"visible": [True, False, False, False, False]},
               {"title": "Glucose",
                "annotations": []}]),
        dict(label="Ethanol",
             method="update",
             args=[{"visible": [False, True, False, False, False]},
               {"title": "Ethanol",
               "annotations": []}]),
       dict(label="Cell mass",
            method="update",
            args=[{"visible": [False, False, True, False, False]},
               {"title": "Cell Mass",
               "annotations": []}]),
       dict(label="Oxygen",
           method="update",
           args=[{"visible": [False, False, False, True, False]},
               {"title": "Oxygen",
               "annotations": []}]),
       dict(label="Carbondioxide",
           method="update",
           args=[{"visible": [False, False, False, False, True]},
               {"title": "Carbondioxide",
               "annotations": []}]),
       ]

     # Add slider configuration with buttons in two different positions
    fig.update_layout(
        updatemenus=[
        {
            'buttons': buttons_left,
            'direction': 'left',
            'pad': {'r': 0, 't': 15},
            'showactive': True,
            'type': 'buttons',
            'x': 0.1,
            'xanchor': 'right',
            'y': 0,
            'yanchor': 'top'
        },
        {
            'buttons': buttons_top,
            'direction': 'down',
            'showactive': False,
            'x': 0.1,
            'xanchor': 'right',
            'y': 1.1,
            'yanchor': 'top'
        }])

    # Create animation frames
    frames = [go.Frame(data=[go.Scatter(x=tspan[:i + 1] * 60, y=c_Glc[:i + 1], name='Glucose'),
                         go.Scatter(x=tspan[:i + 1] * 60, y=c_Eth[:i + 1], name='Ethanol'),
                         go.Scatter(x=tspan[:i + 1] * 60, y=Xv[:i + 1] + Xd[:i + 1], name='Cell mass'),
                         go.Scatter(x=tspan[:i + 1] * 60, y=p_O2[:i + 1], name='Oxygen'),
                         go.Scatter(x=tspan[:i + 1] * 60, y=p_CO2[:i + 1], name='Carbondioxide')],
                   traces=[0, 1, 2, 3, 4],
                   name=f'frame_{i}')
          for i in range(1, len(tspan) + 1)]
    fig.frames = frames
        
    fig.update_xaxes(range=[tspan[0] * 60, tspan[-1] * 60])
    
    fig.update_layout(
        title='Behaviour over Time',
        xaxis_title='Time (minutes)',
        yaxis_title='Concentration (g/L)',
        legend=dict(x=1, y=1),
        width=1265,  
        height=600,
        title_font_size=25,
    )
        
    # Display the Plotly figure
    fig.show();

In [6]:
Aerobic = False

# Initial values for SW_Glc_aerob and SW_Glc_crab
SW_Glc_aerob = 0 if Aerobic else 1
SW_Glc_crab = 1 if Aerobic else 0

# Create a function to toggle the crab variable and update the values
def toggle_Aerobic(_):
    global Aerobic
    Aerobic = not Aerobic
    SW_Glc_aerob = 0 if Aerobic else 1
    SW_Glc_crab = 1 if Aerobic else 0

In [7]:
t = float(input("Enter simulation time (hours): ")) # example 6

In [8]:
y0 = [Xv_0_widget.value, Xd_0_widget.value, c_Glc_0_widget.value, c_Eth_0_widget.value, p_CO2_0_widget.value, p_O2_0_widget.value, m_H20_0_widget.value, Q_0_widget.value, KR_O2_0_widget.value, KR_E_0_widget.value]

# Define equations
Crab_Tree(y0, t)

# Define time span
tspan = np.linspace(0, t, 450)

# Solve the differential equations
sol = odeint(Crab_Tree, y0, tspan)

# Extract variables
Xv = sol[:, 0]
Xd = sol[:, 1]
c_Glc = sol[:, 2]
c_Eth = sol[:, 3]
p_CO2 = sol[:, 4]
p_O2 = sol[:, 5]
XvXd = Xv + Xd

In [9]:
print('Starting conditions:')

Starting conditions:


In [10]:
# Define update function
def update_plot(c_Eth_0, p_CO2_0, p_O2_0, m_H20_0, Q_0, KR_E_0, KR_O2_0, Xd_0, Xv_0, c_Glc_0):
    # Call Crab_Tree with the updated slider values
    y0 = [Xv_0, Xd_0, c_Glc_0, c_Eth_0, p_CO2_0, p_O2_0, m_H20_0, Q_0, KR_O2_0, KR_E_0]
    t_span = np.linspace(0, t, 450)  # Adjust the time span as needed
    result = odeint(Crab_Tree, y0, t_span)

    # Extract values for the plot
    c_Glc, c_Eth, XvXd, p_CO2, p_O2 = result[:, 2], result[:, 3], result[:, 0] + result[:, 1], result[:, 4], result[:, 5]

    # Update the plot using the new values
    create_plot(t_span, c_Glc, c_Eth, XvXd, p_CO2, p_O2)
    
# Use interactive_output to link sliders to update function
interactive_plot = interactive_output(update_plot,
                                       {'c_Eth_0': c_Eth_0_widget,
                                        'p_CO2_0': p_CO2_0_widget,
                                        'p_O2_0': p_O2_0_widget,
                                        'm_H20_0': m_H20_0_widget,
                                        'Q_0': Q_0_widget,
                                        'KR_E_0': KR_E_0_widget,
                                        'KR_O2_0': KR_O2_0_widget,
                                        'Xd_0': Xd_0_widget,
                                        'Xv_0': Xv_0_widget,
                                        'c_Glc_0': c_Glc_0_widget})

# Display the sliders
display(c_Eth_0_widget, p_CO2_0_widget, p_O2_0_widget, m_H20_0_widget, Q_0_widget, KR_E_0_widget, KR_O2_0_widget, Xd_0_widget, Xv_0_widget, c_Glc_0_widget)

# Display the interactive plot
display(interactive_plot)

FloatSlider(value=0.0, description='ethanol (g/L)')

FloatSlider(value=1.0, description='CO2 (g/L)')

FloatSlider(value=20.0, description='O2 (Vol-%)')

FloatSlider(value=0.0, description='Initial ethanol(g/L)')

FloatSlider(value=0.0, description='Temperature', max=10.0)

...

FloatSlider(value=0.0, description='half-saturation O2 ', max=10.0)

FloatSlider(value=0.0, description='Dead cell mass(g)', max=10.0)

FloatSlider(value=5.0, description='Inoculum (g)')

FloatSlider(value=5.0, description='Substrate (g/L)', max=10.0)

Output()